# EV Charging Simulation - Examples and Usage

This notebook demonstrates the EV charging simulation capabilities implemented in Phase 1 and 2.

## Features

- **Physics-based simulation** of AC and DC charging
- **Vehicle charging curves** for realistic DC fast charging
- **Power limiting** based on charger and vehicle constraints
- **Tariff integration** for cost calculation
- **Preset libraries** of popular vehicles and chargers
- **Visualization** of charging sessions

## Setup and Imports

In [ ]:
# Core simulation functions
from sim import simulate_session, estimate_charging_time
from sim.models import Tariff
from sim.presets import vehicles, chargers

# For visualization
import matplotlib.pyplot as plt
import numpy as np

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Example 1: Simple DC Fast Charging

Let's simulate a typical road trip charging scenario: charging a Tesla Model 3 from 20% to 80% at a Supercharger.

In [ ]:
# Define the scenario
vehicle = vehicles.TESLA_MODEL_3_LR
charger = chargers.TESLA_SUPERCHARGER_V3

print(f"Vehicle: {vehicle.name}")
print(f"  Battery: {vehicle.battery_capacity_kwh} kWh")
print(f"  Max DC power: {vehicle.max_dc_power_kw} kW")
print(f"\nCharger: {charger.name}")
print(f"  Max power: {charger.max_power_kw} kW")
print(f"  Type: {charger.charger_type.value}")

In [ ]:
# Run the simulation
result = simulate_session(
    charger=charger,
    vehicle=vehicle,
    start_soc=20,
    target_soc=80
)

# Display results
print(result.summary())

In [ ]:
# Visualize the charging session
result.plot()
plt.show()

## Example 2: AC Charging at Home

Overnight charging with a home wallbox is a common use case. Let's simulate charging a Volkswagen ID.3 at home.

In [ ]:
# Home charging scenario
vehicle = vehicles.VW_ID3_77KWH
charger = chargers.HOME_WALLBOX_11KW

result = simulate_session(
    charger=charger,
    vehicle=vehicle,
    start_soc=30,
    target_soc=100
)

print(result.summary())
result.plot()
plt.show()

## Example 3: Ultra-Fast Charging with 800V Vehicle

Modern 800V vehicles like the Hyundai Ioniq 5 can charge extremely fast. Let's see it in action at an IONITY charger.

In [ ]:
# Ultra-fast charging
vehicle = vehicles.HYUNDAI_IONIQ5_77KWH
charger = chargers.IONITY_350KW

result = simulate_session(
    charger=charger,
    vehicle=vehicle,
    start_soc=10,
    target_soc=80
)

print(result.summary())
print(f"\n⚡ Charged {result.energy_added_kwh:.1f} kWh in just {result.total_time_minutes:.0f} minutes!")
result.plot()
plt.show()

## Example 4: Tariff Integration and Cost Calculation

Let's add pricing information to understand the cost of charging.

In [ ]:
# Create a tariff (based on typical Portuguese Mobie pricing)
tariff = Tariff(
    id="PT-EDP-EXAMPLE",
    operator="EDP Mobie",
    flat_fee=0.261,      # Connection fee
    energy_rate=0.45,    # Per kWh
    time_rate=0.00,      # No time component
    currency="EUR"
)

print(f"Tariff: {tariff.operator}")
print(f"  Flat fee: €{tariff.flat_fee:.2f}")
print(f"  Energy rate: €{tariff.energy_rate:.2f}/kWh")

In [ ]:
# Simulate with tariff
vehicle = vehicles.KIA_EV6_77KWH
charger = chargers.DC_CCS_150KW

result = simulate_session(
    charger=charger,
    vehicle=vehicle,
    start_soc=20,
    target_soc=80,
    tariff=tariff
)

print(result.summary())

## Example 5: Comparing Charging Strategies

Let's compare different charging strategies for a road trip.

In [ ]:
# Compare different SoC ranges for the same vehicle and charger
vehicle = vehicles.TESLA_MODEL_3_LR
charger = chargers.TESLA_SUPERCHARGER_V3

strategies = [
    (10, 50, "Quick stop (10-50%)"),
    (10, 80, "Standard (10-80%)"),
    (10, 90, "Extended (10-90%)"),
    (20, 80, "Conservative (20-80%)"),
]

print("Comparing charging strategies:\n")
print(f"{'Strategy':<25} {'Time (min)':<12} {'Energy (kWh)':<15} {'Avg Power (kW)'}")
print("-" * 70)

for start, target, name in strategies:
    result = simulate_session(charger, vehicle, start, target)
    print(f"{name:<25} {result.total_time_minutes:<12.1f} {result.energy_added_kwh:<15.1f} {result.average_power_kw:.1f}")

Notice how charging from 10-50% is much faster than 10-90% due to the charging curve tapering at high SoC levels!

## Example 6: Vehicle Comparison

Let's compare how different vehicles charge at the same charger.

In [ ]:
# Compare different vehicles
charger = chargers.DC_CCS_150KW
test_vehicles = [
    vehicles.TESLA_MODEL_3_LR,
    vehicles.VW_ID3_77KWH,
    vehicles.HYUNDAI_IONIQ5_77KWH,
    vehicles.NISSAN_LEAF_62KWH,
]

print(f"Charging comparison at {charger.name} (20% → 80%)\n")
print(f"{'Vehicle':<30} {'Time (min)':<12} {'Avg Power (kW)'}")
print("-" * 55)

for vehicle in test_vehicles:
    result = simulate_session(charger, vehicle, 20, 80)
    print(f"{vehicle.name:<30} {result.total_time_minutes:<12.1f} {result.average_power_kw:.1f}")

## Example 7: Available Presets

Let's see what vehicles and chargers are available in the preset library.

In [ ]:
# List all available vehicles
print("Available Vehicles:\n")
for vehicle_name in vehicles.__all__:
    vehicle = getattr(vehicles, vehicle_name)
    print(f"  • {vehicle.name:<35} {vehicle.battery_capacity_kwh:>5.1f} kWh, {vehicle.max_dc_power_kw:>3.0f} kW DC")

In [ ]:
# List all available chargers
from sim.models.charger import ChargerType

print("Available Chargers:\n")
print("AC Chargers:")
for charger_name in chargers.__all__:
    charger = getattr(chargers, charger_name)
    if charger.charger_type == ChargerType.AC:
        print(f"  • {charger.name:<35} {charger.max_power_kw:>5.1f} kW")

print("\nDC Chargers:")
for charger_name in chargers.__all__:
    charger = getattr(chargers, charger_name)
    if charger.charger_type == ChargerType.DC:
        print(f"  • {charger.name:<35} {charger.max_power_kw:>5.1f} kW")

## Example 8: Quick Time Estimation

For quick planning, you can estimate charging time without running a full simulation.

In [ ]:
# Quick estimation
vehicle = vehicles.BMW_IX_111KWH
charger = chargers.DC_CCS_150KW

estimated_time = estimate_charging_time(charger, vehicle, 20, 80)
print(f"Estimated charging time: {estimated_time:.1f} minutes")

# Compare with full simulation
result = simulate_session(charger, vehicle, 20, 80)
print(f"Actual charging time: {result.total_time_minutes:.1f} minutes")
print(f"Difference: {abs(estimated_time - result.total_time_minutes):.1f} minutes")

## Summary

This notebook demonstrated the key features of the EV charging simulation module:

1. ✅ **DC and AC charging simulation** with realistic power curves
2. ✅ **Tariff integration** for cost calculation
3. ✅ **Multiple vehicle and charger presets**
4. ✅ **Visualization** of charging profiles
5. ✅ **Comparison tools** for different scenarios

## Next Steps

- **Phase 3**: Integration with real Mobie tariff data from CSV files
- **Phase 4**: Statistical analysis and Monte Carlo simulation for price distributions

For more information, see the implementation in the `sim/` package.